In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
path="../../Downloads/"

In [3]:
train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
allData = pd.concat([train,test])

In [4]:
## word embedding start
from keras.preprocessing.text import Tokenizer
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embedding(file):
    if file == path+'embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index

def make_embedding_matrix(embedding, tokenizer, len_voc,is_zero = False):
    all_embs = np.stack(embedding.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    if is_zero:
        embedding_matrix = np.zeros((len_voc, embed_size))
    else:
        embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embedding.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

def make_tokenizer(texts, len_voc):
    t = Tokenizer(num_words=len_voc)
    t.fit_on_texts(texts)
    return t

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# tokenizer = make_tokenizer(allData.question_text.values,15000)

In [6]:
## word2vec

In [7]:
file = path+'embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
word2vec = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)

In [8]:
### sentence2vec
allData.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [9]:
#simple average

In [10]:
def sen2vec1(data):
    scores=[]
    for sen in data.question_text:
        words=sen.split()
        tmp=[]
        for wd in words:
            if wd in word2vec:
                tmp.append(word2vec[wd])
        tmp=np.array(tmp)
        if len(tmp)==0:
            scores.append(np.array([0]*300))
        else:
            scores.append(tmp.mean(axis=0))
    data['scores']=scores
                
    
    
    

In [11]:
sen2vec1(allData)

In [12]:
## split trian to trainT, trainV (for tune) and trainTe 8:1:1
X=allData['scores']
Y=allData['target']
X,Xp=X[:len(train)],X[len(train):]
Y=Y[:len(train)]




In [13]:
np.random.seed(1)
unif=np.random.uniform(size=len(train))
Xtr=[]
Ytr=[]
Xte=[]
Yte=[]
Xv=[]
Yv=[]
for i,num in enumerate(unif):
    if num<0.8:
        Xtr.append(X[i])
        Ytr.append(Y[i])
    elif num<0.9:
        Xte.append(X[i])
        Yte.append(Y[i])
    else:
        Xv.append(X[i])
        Yv.append(Y[i])
    

In [14]:
Xtr=np.vstack(Xtr)
Ytr=np.array(Ytr)
Xte=np.vstack(Xte)
Yte=np.array(Yte)
Xv=np.vstack(Xv)
Yv=np.array(Yv)


In [15]:
## simple GBDT
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=200,max_depth=5,verbose=2).fit(Xtr,Ytr)

In [ ]:
clf.score(Xv, Yv)